# TwoPlayer Model #

In [1]:
# Torch and Pyro
import torch._numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer.autoguide import AutoNormal
from pyro.infer import Predictive

# Plotting
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

# Project specific utilities
from utils import *

# Torch and Pyro
import torch
from torch import nn
import torch._numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer.autoguide import AutoNormal
from pyro.infer import Predictive
from pyro.nn import PyroModule

# Plotting
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

# Project specific utilities
from utils import *
from sklearn.model_selection import train_test_split

In [2]:
def TwoPlayer(game_info, obs=None):
    """
    Inputs: 
        game_info: A dictionary containing relevant information about the games played.
        obs: A 1D tensor of observed data. The length should be equal to the number of games played.
             Represents the outcome of a game in the eyes of coach 1, i.e. coach 1 wins: obs = 1, coach 2 wins: obs = -1.

    Output: A 1D tensor of the same length as the input tensor. Represents a sample from difference in performance between the two coaches.
    """
    
    # define hyperparameters
    hyper_sigma = 1/8
    
    coach1_mu_skill = 0
    coach1_sigma_skill = 1

    coach2_mu_skill = 0
    coach2_sigma_skill = 1

    # initialize the skill of the coaches
    coach1_skill = pyro.sample("coach1_skill", dist.Normal(coach1_mu_skill, coach1_sigma_skill))
    coach2_skill = pyro.sample("coach2_skill", dist.Normal(coach2_mu_skill, coach2_sigma_skill))
    
    # define plate for the number of games played
    with pyro.plate('matches', obs.shape[0]):   

        # sample the performance of each coach
        coach1_perf = pyro.sample('coach1_perf', dist.Normal(coach1_skill, hyper_sigma))
        coach2_perf = pyro.sample('coach2_perf', dist.Normal(coach2_skill, hyper_sigma))

        # calculate the difference in performance
        perf_diff = coach1_perf - coach2_perf
    
        # sample the outcome of the game
        y = pyro.sample("y_coach1_win", dist.Normal(perf_diff, hyper_sigma), obs=obs)

    return y

In [3]:
num_games = 100
game_info = {}
obs = torch.ones(num_games)

guide = AutoNormal(TwoPlayer)
losses = run_inference(TwoPlayer, guide, game_info, obs)

predictive = Predictive(TwoPlayer, guide=guide, return_sites=("coach1_skill", "coach2_skill"))
samples = predictive(game_info, obs)

Loss = -23.483765: 100%|██████████| 2000/2000 [00:18<00:00, 108.56it/s]


In [4]:
# Plot the results
fig = go.Figure()
fig.add_trace(go.Histogram(x=samples["coach1_skill"].detach().squeeze(), histnorm='probability density', name="Coach 1"))
fig.add_trace(go.Histogram(x=samples["coach2_skill"].detach().squeeze(), histnorm='probability density', name="Coach 2"))
fig.update_layout(barmode='overlay', xaxis_title="Skill", yaxis_title="Density", title="Coach skill", width=600)
fig.show()

In [5]:
obs = torch.zeros(num_games)
guide = AutoNormal(TwoPlayer)
losses = run_inference(TwoPlayer, guide, game_info, obs)

predictive = Predictive(TwoPlayer, guide=guide, return_sites=("coach1_skill", "coach2_skill"))
samples = predictive(game_info, obs)

Loss = -29.183366: 100%|██████████| 2000/2000 [00:16<00:00, 122.33it/s]


In [6]:
# Plot the results
fig = go.Figure()
fig.add_trace(go.Histogram(x=samples["coach1_skill"].detach().squeeze(), histnorm='probability density', name="Coach 1", opacity=0.75))
fig.add_trace(go.Histogram(x=samples["coach2_skill"].detach().squeeze(), histnorm='probability density', name="Coach 2", opacity=0.75))
fig.update_layout(barmode='overlay', xaxis_title="Skill", yaxis_title="Density", title="Coach skill", width=600)
fig.show()

# Multiplayer Models

In [7]:
default_mu_skill = 0
default_sigma_skill = 1

hyper_sigma = 1/8

In [8]:
def MultiPlayer(game_info, obs=None):
    """
    Inputs:
        game_info: A dictionary containing the following keys:
            coach1_ids: A 1D tensor of length "num_games". The i-th element represents the id of the coach for coach 1 in the i-th game.
            coach2_ids: A 1D tensor of length "num_games". The i-th element represents the id of the coach for coach 2 in the i-th game.
            num_coaches: An integer representing the number of coaches in the data.
        obs: obs: A 1D tensor of observed data. The length should be equal to the number of games played.
             Represents the outcome of a game in the eyes of coach 1, i.e. coach 1 wins: obs = 1, coach 2 wins: obs = -1.

    Output: A 1D tensor of the same length as "obs". Represents a sample from difference in performance between the two coaches.
    """

    # Extract the data
    ids1 = game_info['coach1_ids']
    ids2 = game_info['coach2_ids']
    num_coaches = game_info['num_coaches']
    N = len(ids1)
    
    try:
        coach_mu_skill = game_info['coach_mu_skill']
        coach_sigma_skill = game_info['coach_sigma_skill']
    except:
        coach_mu_skill = default_mu_skill
        coach_sigma_skill = default_sigma_skill

    # Sample skills for each coach using a plate for coaches
    with pyro.plate('coaches', num_coaches):
        coach_skills = pyro.sample("coach_skills", dist.Normal(coach_mu_skill, coach_sigma_skill))
    
    # Sample the performance difference for each match
    with pyro.plate('matches', N):
        # Gather the skills for the competing coaches in each match
        coach1_skills = coach_skills[ids1]
        coach2_skills = coach_skills[ids2]

        # Sample performances for the coaches in each match
        coach1_perf = pyro.sample('coach1_perf', dist.Normal(coach1_skills, hyper_sigma))
        coach2_perf = pyro.sample('coach2_perf', dist.Normal(coach2_skills, hyper_sigma))

        # Compute the performance difference
        perf_diff = coach1_perf - coach2_perf

        # Sample the observed outcomes
        y = pyro.sample("y_coach1_win", dist.Normal(perf_diff, hyper_sigma/2), obs=obs)

    return y

In [9]:
class RaceNet(PyroModule):
    def __init__(self, input_dim, hidden_dim=16, output_dim=1):
        super(RaceNet, self).__init__()

        self.net = nn.Sequential(
            PyroModule[nn.Linear](input_dim, hidden_dim, bias=False),
            nn.Tanh(),
            PyroModule[nn.Linear](hidden_dim, hidden_dim, bias=False),
            nn.Tanh(),
            PyroModule[nn.Linear](hidden_dim, output_dim, bias=False),
        )

    def forward(self, races1, races2): 
        x = races1-races2
        return self.net(x)

num_races = 28
NN = RaceNet(num_races)

def RaceNetMultiPlayer(game_info, obs=None):
    """
    Inputs:
        obs: obs: A 1D tensor of observed data. The length should be equal to the number of games played.
             Represents the outcome of a game in the eyes of coach 1, i.e. coach 1 wins: obs = 1, coach 2 wins: obs = -1.
        ids1: A 1D tensor of the same length as obs. Contains the index of the first coach in each game.
        ids2: A 1D tensor of the same length as obs. Contains the index of the second coach in each game.
        num_coaches: The number of coaches in the dataset.

    Output: A 1D tensor of the same length as "obs". Represents a sample from difference in performance between the two coaches.
    """

    # Extract the game information
    ids1 = game_info['coach1_ids']
    ids2 = game_info['coach2_ids']
    races1 = game_info['races1']
    races2 = game_info['races2']
    num_coaches = game_info['num_coaches']    
    
    try:
        coach_mu_skill = game_info['coach_mu_skill']
        coach_sigma_skill = game_info['coach_sigma_skill']
    except:
        coach_mu_skill = default_mu_skill
        coach_sigma_skill = default_sigma_skill

    num_games = len(ids1)
    num_races = len(races1[0])

    # Sample skills for each coach using a plate for coaches
    with pyro.plate('coaches', num_coaches):
        coach_skills = pyro.sample("coach_skills", dist.Normal(coach_mu_skill, coach_sigma_skill))
    
    with pyro.plate('matches', num_games):
        # Gather the skills for the competing coaches in each match
        coach1_skills = coach_skills[ids1]
        coach2_skills = coach_skills[ids2]

        # Get racial bias
        biases = NN(races1, races2).squeeze()

        # Sample performances for the coaches in each match
        coach1_perf = pyro.sample('coach1_perf', dist.Normal(coach1_skills , hyper_sigma))
        coach2_perf = pyro.sample('coach2_perf', dist.Normal(coach2_skills, hyper_sigma))

        # Compute the performance difference
        perf_diff = coach1_perf - coach2_perf + biases

        # Sample the observed outcomes
        y = pyro.sample("y_coach1_win", dist.Normal(perf_diff, hyper_sigma/2), obs=obs)

    return y

### Sanity check of multiple player TrueSkill model on synthetic data ###

In [10]:
# Generate round-robin data
num_coaches_rr = 6
pairs = torch.tensor([[i, j] for i in range(num_coaches_rr) for j in range(i+1, num_coaches_rr)])
id1 = pairs[:, 0]
id2 = pairs[:, 1]
obs = -torch.ones(len(pairs))

game_info = {'coach1_ids': id1, 'coach2_ids': id2, 'num_coaches': num_coaches_rr}

# Train the model
guide = AutoNormal(MultiPlayer)
losses = run_inference(MultiPlayer, guide, game_info, obs)

# Run predictive posterior
predictive = Predictive(MultiPlayer, guide=guide, num_samples=2000)
samples = predictive(game_info, obs)

# Plot the results
fig = go.Figure()
for i in range(num_coaches_rr):
    hist_data = samples["coach_skills"].detach().squeeze()[:,i]
    fig.add_trace(go.Histogram(x=hist_data, histnorm='probability density'))
    fig.add_annotation(x=hist_data.mean(), y=1, text=f"<b>Coach {i+1}</b>", showarrow=False, font=dict(size=12, color="black"),
        borderpad=4, bgcolor="white", opacity=0.8)
fig.update_layout(barmode='overlay', showlegend=False, xaxis_title="Skill", yaxis_title="Density", title="Coach skill", width=1200)
fig.show()

Loss = 61.164278: 100%|██████████| 2000/2000 [00:14<00:00, 136.37it/s]


## Infer models on real data

### Load Data

In [11]:
data = load_data('../data/df_matches_clean.csv')

all_data = data['all_data']
id1 = data['coach1_id']
id2 = data['coach2_id']
num_coaches = data['num_coaches']
race1 = data['race1']
race2 = data['race2']
races = data['races']
obs = data['obs']
N = len(id1)

# Get indeces for train and test set
train_test_ratio = 0.95
split = int(train_test_ratio*N)
idx_train = np.arange(split)
idx_test = np.arange(split, N)

# Make training set
id1_train = id1[idx_train]
id2_train = id2[idx_train]
obs_train = obs[idx_train]
race1_train = race1[idx_train]
race2_train = race2[idx_train]
game_info_train = {'coach1_ids': id1_train, 'coach2_ids': id2_train, 'num_coaches': num_coaches, 'races1': race1_train, 'races2': race2_train}

# Make test set
id1_test = id1[idx_test]
id2_test = id2[idx_test]
obs_test = obs[idx_test]
race1_test = race1[idx_test]
race2_test = race2[idx_test]
game_info_test = {'coach1_ids': id1_test, 'coach2_ids': id2_test, 'num_coaches': num_coaches, 'races1': race1_test, 'races2': race2_test}

### MultiPlayer Model

In [12]:
# Infer Model
guide_MultiPlayer = AutoNormal(MultiPlayer)
losses_MultiPlayer = run_inference(MultiPlayer, guide_MultiPlayer, game_info_train, obs_train, num_steps=1000)

# Predictive distribution
predictive_MultiPlayer = Predictive(MultiPlayer, guide=guide_MultiPlayer, num_samples=1000)
samples_MultiPlayer = predictive_MultiPlayer(game_info_train, obs_train)

# Predict training data
game_info_train['coach_mu_skill'] = samples_MultiPlayer["coach_skills"].mean(dim=0)
game_info_train['coach_sigma_skill'] = samples_MultiPlayer["coach_skills"].std(dim=0)
game_info_test['coach_mu_skill'] = game_info_train['coach_mu_skill']
game_info_test['coach_sigma_skill'] = game_info_train['coach_sigma_skill']

train_prediction_MultiPlayer = MultiPlayer(game_info_train).detach().numpy()
test_prediction_MultiPlayer = MultiPlayer(game_info_test).detach().numpy()

Loss = 749988.717285: 100%|██████████| 1000/1000 [00:27<00:00, 35.87it/s]


### RaceNetMultiPlayer Model

In [13]:
# Infer RaceNetModel
guide_RaceNetMultiPlayer = AutoNormal(RaceNetMultiPlayer)
losses_RaceNetMultiPlayer = run_inference(RaceNetMultiPlayer, guide_RaceNetMultiPlayer, game_info_train, obs_train, num_steps=1000)

# Predictive distribution
predictive_RaceNetMultiPlayer = Predictive(RaceNetMultiPlayer, guide=guide_RaceNetMultiPlayer, num_samples=1000)
samples_RaceNetMultiPlayer = predictive_RaceNetMultiPlayer(game_info_train, obs_train)

# Predict
game_info_train['coach_mu_skill'] = samples_RaceNetMultiPlayer["coach_skills"].mean(dim=0)
game_info_train['coach_sigma_skill'] = samples_RaceNetMultiPlayer["coach_skills"].std(dim=0)
game_info_train['coach_mu_skill'] = game_info_train['coach_mu_skill']
game_info_train['coach_sigma_skill'] = game_info_train['coach_sigma_skill']

train_prediction_RaceNetMultiPlayer = RaceNetMultiPlayer(game_info_train).detach().numpy()
test_prediction_RaceNetMultiPlayer = RaceNetMultiPlayer(game_info_test).detach().numpy()

Loss = 715144.632324: 100%|██████████| 1000/1000 [00:43<00:00, 22.93it/s]


### Distribution and Trueskill samples of coaches

In [22]:
# Get a sample of coaches that played more than 10 matches
mask = data['sorted_num_matches'] > 10
eligible_coaches = np.where(mask)[0]
selected_coaches = np.random.choice(eligible_coaches, 10, replace=False)

fig = go.Figure()
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

# Get a random coach that has only played one match and won
one_game_won_coaches = [i for i, (winrate, num_matches) in enumerate(zip(data['sorted_winrates'], data['sorted_num_matches'])) if winrate == 1.0 and num_matches == 1]
selected_coach = np.random.choice(one_game_won_coaches)
selected_coaches[-1] = selected_coach

for idx, i in enumerate(selected_coaches):
    samples = samples_MultiPlayer["coach_skills"].detach().squeeze()[:,i]
    fig.add_trace(go.Histogram(x=samples, name=f"winrate: {data['sorted_winrates'][i]:.2f}, num_matches: {int(data['sorted_num_matches'][i])}",
                               histnorm='probability density', marker_color=colors[idx],opacity=0.8))

    # Calculate mean and standard deviation
    mean = samples.mean().item()
    std_dev = samples.std().item()

    # Add vertical lines for 3 standard deviations from the mean
    fig.add_shape(go.layout.Shape(type="line", x0=mean-3*std_dev, x1=mean-3*std_dev, y0=0, y1=0.1, yref='paper', line=dict(color=colors[idx], width=4)))

    # Add a black line over the colored line
    fig.add_shape(go.layout.Shape(type="line", x0=mean-3*std_dev, x1=mean-3*std_dev, y0=0, y1=0.1, yref='paper', line=dict(color='black', width=1)))

fig.update_layout(barmode='overlay', xaxis_title="Skill", yaxis_title="Density", title="Coach skill", width=1200)
fig.show()

# Investigate FFNN vs bias as winrate 

## Compute winrate for each pairing of races ##

In [23]:
winrates = np.zeros((len(races), len(races)))

for i, race1 in enumerate(races):
    for j, race2 in enumerate(races):
        win_as_p1 = sum((all_data['team1_win'] == 1) & (all_data['team1_race_name'] == race1) & (all_data['team2_race_name'] == race2))
        win_as_p2 = sum((all_data['team2_win'] == 1) & (all_data['team2_race_name'] == race1) & (all_data['team1_race_name'] == race2))
        draws = sum((all_data['team1_win'] == 0) & (all_data['team1_race_name'] == race1) & (all_data['team2_race_name'] == race2))
        total = sum((all_data['team1_race_name'] == race1) & (all_data['team2_race_name'] == race2)) + sum((all_data['team2_race_name'] == race1) & (all_data['team1_race_name'] == race2))
        winrates[j, i] = (win_as_p1 + win_as_p2 + draws) / total

## Neural Network Racial Heatmap

In [24]:
# Construct the heatmap
heatmap = torch.zeros(num_races, num_races)
for i in range(num_races):
    for j in range(num_races):
        r1 = torch.zeros(1, num_races)
        r1[0, i] = 1
        r2 = torch.zeros(1, num_races)
        r2[0, j] = 1
        heatmap[j, i] = NN(r1, r2)[0, 0]

heatmap = heatmap.detach().numpy()

# Plot the heatmap
fig = go.Figure(data=go.Heatmap(z=heatmap, x=races, y=races, colorscale='RdBu_r',))


# Set the labels
fig.update_xaxes(title_text='Player', tickangle=90)
fig.update_yaxes(title_text='Opponent', tickangle=0)

# Set the title
fig.update_layout(title_text='Racial performance bias', autosize=False, width=800, height=800)
# Show the plot
fig.show()

# Calculate the mean and reshape it into a 1D array
mean_values = np.reshape(heatmap.mean(axis=0), (-1,))
color_scale = np.interp(mean_values, (mean_values.min(), mean_values.max()), [-1, 1])


# Create the bar plot
fig = go.Figure(data=go.Bar(x=races, y=mean_values, marker=dict(color=color_scale, colorscale='RdBu_r',)))
fig.update_layout(title_text='Average Racial Performance Bias', width=800)
fig.update_xaxes(tickangle=45, )
fig.show()

## Find Pearson correlation between winrate heatmap and FFNN heatmap

In [25]:
#Normalize before comparing
heatmap_winrate = (winrates - winrates.min()) / (winrates.max() - winrates.min())
heatmap_FFNN = (heatmap  - heatmap.min()) / (heatmap.max() - heatmap.min())

#Find pearson correlation
corr = np.corrcoef(heatmap.flatten(), heatmap_FFNN.flatten())
corrcoef = corr[0, 1]
print(f"pearson correlation: {corrcoef}")

pearson correlation: 0.999999999999995


# Investigate Draw Margin for Multiplayer, RaceNetMultiplayer Model

In [39]:
def accuracy(margin, pred, targ):
    pred = np.select([pred < -margin, abs(pred) <= margin, pred > margin], [-1, 0, 1])
    return np.mean(pred == targ)  

margins = np.linspace(0, 2, 200)
acc = np.zeros((4, len(margins)))
for i, margin in enumerate(margins):
    acc[0, i] = accuracy(margin, train_prediction_MultiPlayer, obs_train.numpy())
    acc[1, i] = accuracy(margin, test_prediction_MultiPlayer, obs_test.numpy())
    acc[2, i] = accuracy(margin, train_prediction_RaceNetMultiPlayer, obs_train.numpy())
    acc[3, i] = accuracy(margin, test_prediction_RaceNetMultiPlayer, obs_test.numpy())

baseline_acc = max([np.mean(obs.numpy() == i) for i in [-1, 0, 1]])

fig = go.Figure()
fig.add_trace( go.Scatter(x=margins, y=acc[0], mode='lines', name='MultiPlayer Train') )
fig.add_trace( go.Scatter(x=margins, y=acc[1], mode='lines', name='MultiPlayer Test') )
fig.add_trace( go.Scatter(x=margins, y=acc[2], mode='lines', name='RaceNetMultiPlayer Train') )
fig.add_trace( go.Scatter(x=margins, y=acc[3], mode='lines', name='RaceNetMultiPlayer Test') )
fig.add_trace( go.Scatter(x=margins, y=[baseline_acc] * len(margins), mode='lines', name='Baseline') )
fig.update_layout(title='Model Accuracy on training data', xaxis_title='Draw Margin', yaxis_title='Accuracy', width=800, height=600)
fig.update_yaxes(range=[0, 1])
fig.show()